# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [1]:
import findspark
findspark.init()

## Spark Session creation


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://672e28abb623:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d2e0a20-db3c-49f2-bdfd-30915bf7f294;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

## Kafka Stream creation

In [3]:
streamer_lines = []

for i in range(4):
    streamer_lines.append( spark \
                            .readStream \
                            .format("kafka") \
                            .option("kafka.bootstrap.servers", "dee5c9cc3710:9093") \
                            .option("subscribe", f"stock_topic{i}") \
                            .load()
    )


## Transform binary data into string

In [4]:
streamer_df = []
for i in range(4):
    streamer_df.append( streamer_lines[i].withColumn("value_str", streamer_lines[i] \
                .value.cast("string")) \
                .select("timestamp", "value_str")
    )


### Sink configuration

In [ ]:

query = []
for i in range(4):
    query.append( streamer_df[i] \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='8 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()
    )

    #query[i].awaitTermination(100)


25/05/10 02:40:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f6146827-4675-41b8-bf34-a2c0c9b185f4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/10 02:40:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/10 02:40:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-eba098c8-58d0-47d1-b2bb-eec3051aa06f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/10 02:40:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

25/05/10 02:40:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/10 02:40:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/10 02:40:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/10 02:40:06 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
-------------------------------------------
Batch: 0
-------------------------------------------
+---------+---------+
|timestamp|value_str|
+---------+---------+
+---------+---------+

+---------+---------+
|timestamp|value_str|
+---------+---------+
+---------+---------+

+---------+---------+
|timestamp|value_str|
+---------+---------+
+---------+---------+

+---------+---------+
|timestamp|value_str|
+---------+---------+
+---------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------


-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+--------------------------------------------------------------------------------------------------------+
|timestamp              |value_str                                                                                               |
+-----------------------+--------------------------------------------------------------------------------------------------------+
|2025-05-10 02:40:07.567|2025-05-10 03:05:00+00:00,AAPL,198.09395871770963,198.2372315452369,198.06132652775867,198.2372315452369|
+-----------------------+--------------------------------------------------------------------------------------------------------+

+-----------------------+-------------------------------------------------------------------------------------------------------+
|timestamp              |value_str                                                                                   

-------------------------------------------
Batch: 2
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+----------------------------------------------------------------------------------------------------------+
|timestamp              |value_str                                                                                                 |
+-----------------------+----------------------------------------------------------------------------------------------------------+
|2025-05-10 02:40:09.568|2025-05-10 03:10:00+00:00,AAPL,198.22944046304715,198.28874696940787,198.1346426607149,198.18115951495915 |
|2025-05-10 02:40:11.569|2025-05-10 03:15:00+00:00,AAPL,198.16040703190322,198.26483338025074,197.94170371815474,198.071386557122  |
|2025-05-10 02:40:13.571|2025-05-10 03:20:00+00:00,AAPL,198.05278983446564,198.32732263623288,198.00494300497186,198.2580994060082 |
|2025-05

-------------------------------------------
Batch: 2
-------------------------------------------
+-----------------------+---------------------------------------------------------------------------------------------------------+
|timestamp              |value_str                                                                                                |
+-----------------------+---------------------------------------------------------------------------------------------------------+
|2025-05-10 02:40:10.456|2025-05-10 03:10:00+00:00,CVX,139.69204514480325,139.7939860946401,139.68562756214902,139.7477518534348  |
|2025-05-10 02:40:12.457|2025-05-10 03:15:00+00:00,CVX,139.76370140450638,139.8006495829252,139.67035919166975,139.69288787132365 |
|2025-05-10 02:40:14.46 |2025-05-10 03:20:00+00:00,CVX,139.69051420688595,139.74057552836697,139.66041439807813,139.74057552836697|
+-----------------------+--------------------------------------------------------------------------------------

In [6]:
for i in range(4):
    query[i].stop()

In [7]:
sc.stop()